This notebook is based on my previous notebook [Baseline_For_Beginners](https://www.kaggle.com/code/adamlogman/baseline-for-beginners),and [Dmitry Uarov](https://www.kaggle.com/dmitryuarov) 's [NeurIPS | Baseline + External data](https://www.kaggle.com/code/dmitryuarov/neurips-baseline-external-data) notebook with modifications to the model

# Import Dependencies 

In [1]:
!pip install /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl
!pip install mordred --no-index --find-links=file:///kaggle/input/mordred-1-2-0-py3-none-any/

Processing /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl
Looking in links: file:///kaggle/input/mordred-1-2-0-py3-none-any/
Processing /kaggle/input/mordred-1-2-0-py3-none-any/mordred-1.2.0-py3-none-any.whl
Processing /kaggle/input/mordred-1-2-0-py3-none-any/networkx-2.8.8-py3-none-any.whl (from mordred)
  Attempting uninstall: networkx
    Found existing installation: networkx 3.4.2
    Uninstalling networkx-3.4.2:
      Successfully uninstalled networkx-3.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.25.2 requires networkx>=3.0, but you have networkx 2.8.8 which is incompatible.
nx-cugraph-cu12 25.2.0 requires networkx>=3.2, but you have networkx 2.8.8 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but yo

In [2]:
 # Importing Required Libraries\nLet's begin by importing the essential Python libraries needed for data processing, visualization, and modeling.

import pandas as pd
import numpy as np


from sklearn.ensemble import HistGradientBoostingRegressor,ExtraTreesRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from mordred import Calculator, descriptors

import networkx as nx
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdmolops
from rdkit import Chem

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [3]:
class CFG:
    TARGETS = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']
    SEED = 42
    FOLDS = 10

In [4]:
useless_cols = [   
    
    'MaxPartialCharge', 
    # Nan data
    'BCUT2D_MWHI',
    'BCUT2D_MWLOW',
    'BCUT2D_CHGHI',
    'BCUT2D_CHGLO',
    'BCUT2D_LOGPHI',
    'BCUT2D_LOGPLOW',
    'BCUT2D_MRHI',
    'BCUT2D_MRLOW',

    # Constant data
    'NumRadicalElectrons',
    'SMR_VSA8',
    'SlogP_VSA9',
    'fr_barbitur',
    'fr_benzodiazepine',
    'fr_dihydropyridine',
    'fr_epoxide',
    'fr_isothiocyan',
    'fr_lactam',
    'fr_nitroso',
    'fr_prisulfonamd',
    'fr_thiocyan',

    # High correlated data >0.95
    'MaxEStateIndex',
    'HeavyAtomMolWt',
    'ExactMolWt',
    'NumValenceElectrons',
    'Chi0',
    'Chi0n',
    'Chi0v',
    'Chi1',
    'Chi1n',
    'Chi1v',
    'Chi2n',
    'Kappa1',
    'LabuteASA',
    'HeavyAtomCount',
    'MolMR',
    'Chi3n',
    'BertzCT',
    'Chi2v',
    'Chi4n',
    'HallKierAlpha',
    'Chi3v',
    'Chi4v',
    'MinAbsPartialCharge',
    'MinPartialCharge',
    'MaxAbsPartialCharge',
    'FpDensityMorgan2',
    'FpDensityMorgan3',
    'Phi',
    'Kappa3',
    'fr_nitrile',
    'SlogP_VSA6',
    'NumAromaticCarbocycles',
    'NumAromaticRings',
    'fr_benzene',
    'VSA_EState6',
    'NOCount',
    'fr_C_O',
    'fr_C_O_noCOO',
    'NumHDonors',
    'fr_amide',
    'fr_Nhpyrrole',
    'fr_phenol',
    'fr_phenol_noOrthoHbond',
    'fr_COO2',
    'fr_halogen',
    'fr_diazo',
    'fr_nitro_arom',
    'fr_phos_ester'
]

SEED = 42

# Read Files

### Main Files

In [5]:
 #We will load both the training and test datasets using pandas, and store test IDs 
train=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train.csv')
test=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/test.csv')
ss=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/sample_submission.csv')
ID=test['id'].copy()

In [6]:
mols_test = [Chem.MolFromSmiles(s) for s in test.SMILES]

# Initialize the Mordred Calculator
calc = Calculator(descriptors, ignore_3D=True) # ignore_3D=True for 2D descriptors

desc_test = calc.pandas(mols_test)
desc_test.head()

100%|██████████| 3/3 [00:00<00:00,  5.37it/s]


ABC  \
0  module 'numpy' has no attribute 'float'.\n`np....   
1  module 'numpy' has no attribute 'float'.\n`np....   
2  module 'numpy' has no attribute 'float'.\n`np....   

                                               ABCGG  nAcid  nBase    SpAbs_A  \
0  module 'numpy' has no attribute 'float'.\n`np....      0      0  51.369068   
1  module 'numpy' has no attribute 'float'.\n`np....      0      0  53.698919   
2  module 'numpy' has no attribute 'float'.\n`np....      0      0  60.526875   

    SpMax_A  SpDiam_A     SpAD_A   SpMAD_A   LogEE_A     VE1_A     VE2_A  \
0  2.608687  5.217373  51.369068  1.252904  4.633213  4.054325  0.098886   
1  2.429583  4.859165  53.698919  1.309730  4.643161  4.685471  0.114280   
2  2.564154  4.983326  60.526875  1.315802  4.764144  4.942681  0.107450   

      VE3_A          VR1_A         VR2_A      VR3_A  nAromAtom  nAromBond  \
0  2.810771  138187.404971   3370.424511  13.247353         24         24   
1  2.955453     981.187389     23.931400   8.299750         30         30   
2  3.123964  637481.849542  13858.301077  14.891337         24         24   

   nAtom  nHeavyAtom  nSpiro  nBridgehead  nHetero  nH  nB  nC  nN  nO  nS  \
0     59          41       0            0       12  18   0  29   2   2   0   
1     67          41       0            0        6  26   0  35   0   4   0   
2     76          46       0            0       10  30   0  36   2   6   0   

   nP  nF  nCl  nBr  nI  nX  ATS0dv  ATS1dv  ATS2dv  ATS3dv  ATS4dv  ATS5dv  \
0   0   6    0    0   0   6   754.0   647.0  1100.0  1133.0  1481.0   988.0   
1   0   0    0    0   0   0   534.0   554.0   760.0   789.0   700.0   672.0   
2   0   0    0    0   0   0   648.0   655.0   974.0  1023.0   953.0   805.0   

   ATS6dv  ATS7dv  ATS8dv  ATS0d  ATS1d  ATS2d  ATS3d  ATS4d  ATS5d  ATS6d  \
0   788.0   662.0   564.0  234.0  291.0  429.0  490.0  466.0  427.0  353.0   
1   648.0   576.0   550.0  244.0  303.0  463.0  534.0  512.0  485.0  465.0   
2   797.0   722.0   743.0  276.0  355.0  550.0  599.0  573.0  552.0  534.0   

   ATS7d  ATS8d                                            ATS0s  \
0  311.0  294.0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  414.0  361.0  missing intrinsic state for ['*'] (ATS0s/Props)   
2  515.0  470.0  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS1s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS2s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS3s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS4s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS5s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS6s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS7s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS8s   ATS0Z   ATS1Z   ATS2Z  \
0  missing intrinsic state for ['*'] (ATS0s/Props)  1774.0  1789.0  2888

In [7]:
train.head()

,id,SMILES,Tg,FFV,Tc,Density,Rg
0,87817,*CC(*)c1ccccc1C(=O)OCCCCCC,NaN,0.374645,0.205667,NaN,NaN
1,106919,*Nc1ccc([C@H](CCC)c2ccc(C3(c4ccc([C@@H](CCC)c5...,NaN,0.370410,NaN,NaN,NaN
2,388772,*Oc1ccc(S(=O)(=O)c2ccc(Oc3ccc(C4(c5ccc(Oc6ccc(...,NaN,0.378860,NaN,NaN,NaN
3,519416,*Nc1ccc(-c2c(-c3ccc(C)cc3)c(-c3ccc(C)cc3)c(N*)...,NaN,0.387324,NaN,NaN,NaN
4,539187,*Oc1ccc(OC(=O)c2cc(OCCCCCCCCCOCC3CCCN3c3ccc([N...,NaN,0.355470,NaN,NaN,NaN


In [8]:
train.shape, test.shape

((7973, 7), (3, 2))

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7973 entries, 0 to 7972
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       7973 non-null   int64  
 1   SMILES   7973 non-null   object 
 2   Tg       511 non-null    float64
 3   FFV      7030 non-null   float64
 4   Tc       737 non-null    float64
 5   Density  613 non-null    float64
 6   Rg       614 non-null    float64
dtypes: float64(5), int64(1), object(1)
memory usage: 436.2+ KB


### Extra Files

In [10]:
tc_smiles =pd.read_csv('/kaggle/input/tc-smiles/Tc_SMILES.csv',      low_memory=False)
tg_smiles =pd.read_csv('/kaggle/input/smiles-extra-data/JCIM_sup_bigsmiles.csv',      low_memory=False)
ktg_smiles =pd.read_excel('/kaggle/input/smiles-extra-data/data_tg3.xlsx')
de_smiles =pd.read_excel('/kaggle/input/smiles-extra-data/data_dnst1.xlsx')
ds1_smiles=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset1.csv',      low_memory=False)
ds2_smiles=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset2.csv',      low_memory=False)
ds3_smiles=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset3.csv',      low_memory=False)
ds4_smiles=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv',      low_memory=False)

tg_ds      = pd.read_csv('/kaggle/input/modred-dataset/desc_tg.csv',      low_memory=False)
tc_ds      = pd.read_csv('/kaggle/input/modred-dataset/desc_tc.csv',      low_memory=False)
rg_ds      = pd.read_csv('/kaggle/input/modred-dataset/desc_rg.csv',      low_memory=False)
ffv_ds     = pd.read_csv('/kaggle/input/modred-dataset/desc_ffv.csv',     low_memory=False)
den_ds     = pd.read_csv('/kaggle/input/modred-dataset/desc_de.csv',      low_memory=False)

# Preprocessing 

In [11]:
def _clean_train(df: pd.DataFrame) -> pd.DataFrame:
    const_cols = [c for c in df.columns if df[c].nunique() <= 1]
    if const_cols:
        df = df.drop(columns=const_cols)
    return df.select_dtypes(include=[np.number]).copy()

tg_ds      = _clean_train(tg_ds)
tc_ds      = _clean_train(tc_ds)
rg_ds      = _clean_train(rg_ds)
ffv_ds     = _clean_train(ffv_ds)
den_ds     = _clean_train(den_ds)
for i in (tg_ds,tc_ds,rg_ds,ffv_ds,den_ds):
     i.drop(columns=[col for col in i.columns if i[col].nunique() == 1],axis=1,inplace=True)


In [12]:
tg_ds.head()

,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,VE1_A,VE2_A,VE3_A,VR1_A,VR2_A,nAromAtom,nAromBond,nAtom,nHeavyAtom,nSpiro,nBridgehead,nHetero,nH,nB,nC,nN,nO,nS,nP,nF,nCl,nBr,nI,nX,ATS0dv,ATS1dv,ATS2dv,ATS3dv,ATS4dv,ATS5dv,ATS6dv,ATS7dv,ATS8dv,ATS0d,ATS1d,ATS2d,ATS3d,ATS4d,ATS5d,ATS6d,ATS7d,ATS8d,ATS0Z,ATS1Z,ATS2Z,ATS3Z,ATS4Z,ATS5Z,ATS6Z,ATS7Z,ATS8Z,AATS0dv,AATS1dv,AATS2dv,AATS0d,AATS1d,AATS2d,AATS0Z,AATS1Z,AATS2Z,ATSC0dv,ATSC1dv,ATSC2dv,ATSC3dv,ATSC4dv,ATSC5dv,ATSC6dv,ATSC7dv,ATSC8dv,ATSC0d,ATSC1d,ATSC2d,ATSC3d,ATSC4d,ATSC5d,ATSC6d,ATSC7d,ATSC8d,ATSC0Z,ATSC1Z,ATSC2Z,ATSC3Z,ATSC4Z,ATSC5Z,ATSC6Z,ATSC7Z,ATSC8Z,AATSC0dv,AATSC1dv,AATSC2dv,AATSC0d,AATSC1d,AATSC2d,AATSC0Z,AATSC1Z,AATSC2Z,MATS1dv,MATS2dv,MATS1Z,MATS2Z,GATS1dv,GATS2dv,GATS1Z,GATS2Z,BCUTdv-1h,BCUTdv-1l,BCUTd-1h,BCUTd-1l,BCUTZ-1h,BCUTZ-1l,BalabanJ,BertzCT,nBonds,nBondsO,nBondsS,nBondsD,nBondsT,nBondsA,nBondsM,nBondsKS,nBondsKD,C1SP1,C2SP1,C1SP2,C2SP2,C3SP2,C1SP3,C2SP3,C3SP3,C4SP3,FCSP3,Xch-3d,Xch-4d,Xch-5d,Xch-6d,Xch-7d,Xch-3dv,Xch-4dv,Xc-3d,Xc-4d,Xc-5d,Xc-6d,Xpc-4d,Xpc-5d,Xpc-6d,Xp-1d,Xp-2d,Xp-3d,Xp-4d,Xp-5d,Xp-6d,Xp-7d,SZ,MZ,SpAbs_D,SpMax_D,SpDiam_D,SpAD_D,SpMAD_D,LogEE_D,VE1_D,VE2_D,VE3_D,VR1_D,VR2_D,NsLi,NsssB,NsCH3,NdCH2,NssCH2,NtCH,NdsCH,NaaCH,NsssCH,NddC,NtsC,NdssC,NaasC,NaaaC,NssssC,NsNH2,NssNH,NaaNH,NtN,NsssNH,NdsN,NaaN,NsssN,NddsN,NaasN,NssssN,NsOH,NdO,NssO,NaaO,NsF,NssSiH2,NsssSiH,NssssSi,NsssP,NdsssP,NsSH,NdS,NssS,NaaS,NdssS,NddssS,NsCl,NssssGe,NssSe,NaaSe,NsBr,NssssSn,NsI,SsLi,SsssB,SsCH3,SdCH2,SssCH2,StCH,SdsCH,SaaCH,SsssCH,SddC,StsC,SdssC,SaasC,SaaaC,SssssC,SsNH2,SssNH,SaaNH,StN,SsssNH,SdsN,SaaN,SsssN,SddsN,SaasN,SssssN,SsOH,SdO,SssO,SaaO,SsF,SssSiH2,SsssSiH,SssssSi,SsssP,SdsssP,SsSH,SdS,SssS,SaaS,SdssS,SddssS,SsCl,SssssGe,SssSe,SaaSe,SsBr,SssssSn,SsI,ECIndex,ETA_alpha,AETA_alpha,ETA_shape_p,ETA_shape_y,ETA_shape_x,ETA_beta,AETA_beta,ETA_beta_s,AETA_beta_s,ETA_beta_ns,AETA_beta_ns,ETA_beta_ns_d,AETA_beta_ns_d,ETA_eta,AETA_eta,ETA_eta_L,AETA_eta_L,ETA_eta_R,AETA_eta_R,ETA_eta_RL,AETA_eta_RL,ETA_eta_F,AETA_eta_F,ETA_eta_FL,AETA_eta_FL,ETA_dAlpha_A,ETA_dAlpha_B,ETA_epsilon_1,ETA_epsilon_2,ETA_epsilon_3,ETA_epsilon_4,ETA_epsilon_5,ETA_dEpsilon_A,ETA_dEpsilon_B,ETA_dEpsilon_C,ETA_dEpsilon_D,ETA_dBeta,AETA_dBeta,ETA_psi_1,ETA_dPsi_A,ETA_dPsi_B,fragCpx,fMF,nHBAcc,nHBDon,IC0,IC1,IC2,IC3,IC4,IC5,TIC0,TIC1,TIC2,TIC3,TIC4,TIC5,SIC0,SIC1,SIC2,SIC3,SIC4,SIC5,BIC0,BIC1,BIC2,BIC3,BIC4,BIC5,CIC0,CIC1,CIC2,CIC3,CIC4,CIC5,MIC0,MIC1,MIC2,MIC3,MIC4,MIC5,ZMIC0,ZMIC1,ZMIC2,ZMIC3,ZMIC4,ZMIC5,FilterItLogS,VMcGowan,LabuteASA,PEOE_VSA1,PEOE_VSA2,PEOE_VSA3,PEOE_VSA4,PEOE_VSA5,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,PEOE_VSA9,PEOE_VSA10,PEOE_VSA11,PEOE_VSA12,PEOE_VSA13,SMR_VSA1,SMR_VSA2,SMR_VSA3,SMR_VSA4,SMR_VSA5,SMR_VSA6,SMR_VSA7,SMR_VSA9,SlogP_VSA1,SlogP_VSA2,SlogP_VSA3,SlogP_VSA4,SlogP_VSA5,SlogP_VSA6,SlogP_VSA7,SlogP_VSA8,SlogP_VSA10,SlogP_VSA11,EState_VSA1,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,EState_VSA6,EState_VSA7,EState_VSA8,EState_VSA9,EState_VSA10,VSA_EState1,VSA_EState2,VSA_EState3,VSA_EState4,VSA_EState5,VSA_EState6,VSA_EState7,VSA_EState8,VSA_EState9,MID,AMID,MID_h,AMID_h,MID_C,AMID_C,MID_N,AMID_N,MID_O,AMID_O,MID_X,AMID_X,MPC2,MPC3,MPC4,MPC5,MPC6,MPC7,MPC8,MPC9,MPC10,TMPC10,piPC1,piPC2,piPC3,piPC4,piPC5,piPC6,piPC7,piPC8,piPC9,piPC10,TpiPC10,apol,bpol,nRing,n3Ring,n4Ring,n5Ring,n6Ring,n7Ring,n8Ring,n12Ring,nG12Ring,nHRing,n3HRing,n4HRing,n5HRing,n6HRing,n7HRing,n8HRing,n12HRing,nG12HRing,naRing,n3aRing,n5aRing,n6aRing,n7aRing,naHRing,n3aHRing,n5aHRing,n6aHRing,n7aHRing,nARing,n3ARing,n4ARing,n5ARing,n6ARing,n7ARing,n8ARing,n12ARing,nG12ARing,nAHRing,n3AHRing,n4AHRing,n5AHRing,n6AHRing,n7AHRing,n8AHRing,n12AHRing,nG12AHRing,nFRing,n7FRing,n8FRing,n9FRing,n10FRing,n11FRing,n12FRing,nG12FRing,nFHRing,n8FHRing,n9FHRing,n10FHRing,n11FHRing,n12FHRing,nG12FHRing,nFaRing,n8FaRing,n9FaRing,n10FaRing,n12FaRing,nG12FaRing,nFaHRing,n8FaHRing,n9FaHRing,n10FaHRing,n12FaHRing,nG12FaHRing,nFARing,n7FARing,n8FARing,n9FARing,n10FARing,n11FARing,n12FARing,nG12FARing,nFAHRing

In [13]:
tg_ds.shape

(9117, 592)

In [14]:
tc_ds.head()

,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,VE1_A,VE2_A,VE3_A,VR1_A,VR2_A,VR3_A,nAromAtom,nAromBond,nAtom,nHeavyAtom,nBridgehead,nHetero,nH,nC,nN,nO,nS,nF,nCl,nBr,nX,ATS0dv,ATS1dv,ATS2dv,ATS3dv,ATS4dv,ATS5dv,ATS6dv,ATS7dv,ATS8dv,ATS0d,ATS1d,ATS2d,ATS3d,ATS4d,ATS5d,ATS6d,ATS7d,ATS8d,ATS0Z,ATS1Z,ATS2Z,ATS3Z,ATS4Z,ATS5Z,ATS6Z,ATS7Z,ATS8Z,AATS0dv,AATS1dv,AATS2dv,AATS3dv,AATS0d,AATS1d,AATS2d,AATS3d,AATS0Z,AATS1Z,AATS2Z,AATS3Z,ATSC0dv,ATSC1dv,ATSC2dv,ATSC3dv,ATSC4dv,ATSC5dv,ATSC6dv,ATSC7dv,ATSC8dv,ATSC0d,ATSC1d,ATSC2d,ATSC3d,ATSC4d,ATSC5d,ATSC6d,ATSC7d,ATSC8d,ATSC0Z,ATSC1Z,ATSC2Z,ATSC3Z,ATSC4Z,ATSC5Z,ATSC6Z,ATSC7Z,ATSC8Z,AATSC0dv,AATSC1dv,AATSC2dv,AATSC3dv,AATSC0d,AATSC1d,AATSC2d,AATSC3d,AATSC0Z,AATSC1Z,AATSC2Z,AATSC3Z,MATS1dv,MATS2dv,MATS3dv,MATS1d,MATS2d,MATS3d,MATS1Z,MATS2Z,MATS3Z,GATS1dv,GATS2dv,GATS3dv,GATS1d,GATS2d,GATS3d,GATS1Z,GATS2Z,GATS3Z,BCUTdv-1h,BCUTdv-1l,BCUTd-1h,BCUTd-1l,BCUTZ-1h,BCUTZ-1l,BalabanJ,BertzCT,nBonds,nBondsO,nBondsS,nBondsD,nBondsT,nBondsA,nBondsM,nBondsKS,nBondsKD,C1SP1,C2SP1,C1SP2,C2SP2,C3SP2,C1SP3,C2SP3,C3SP3,C4SP3,HybRatio,FCSP3,Xch-3d,Xch-4d,Xch-5d,Xch-6d,Xch-7d,Xch-3dv,Xch-4dv,Xc-3d,Xc-4d,Xc-5d,Xc-6d,Xpc-4d,Xpc-5d,Xpc-6d,Xp-0d,Xp-1d,Xp-2d,Xp-3d,Xp-4d,Xp-5d,Xp-6d,Xp-7d,AXp-0d,AXp-1d,AXp-2d,SZ,MZ,SpAbs_D,SpMax_D,SpDiam_D,SpAD_D,SpMAD_D,LogEE_D,VE1_D,VE2_D,VE3_D,VR1_D,VR2_D,VR3_D,NsCH3,NdCH2,NssCH2,NtCH,NdsCH,NaaCH,NsssCH,NddC,NtsC,NdssC,NaasC,NaaaC,NssssC,NsNH2,NssNH,NaaNH,NtN,NdsN,NaaN,NsssN,NddsN,NaasN,NssssN,NsOH,NdO,NssO,NaaO,NsF,NssS,NaaS,NdssS,NddssS,NsCl,NsBr,SsCH3,SdCH2,SssCH2,StCH,SdsCH,SaaCH,SsssCH,SddC,StsC,SdssC,SaasC,SaaaC,SssssC,SsNH2,SssNH,SaaNH,StN,SdsN,SaaN,SsssN,SddsN,SaasN,SssssN,SsOH,SdO,SssO,SaaO,SsF,SssS,SaaS,SdssS,SddssS,SsCl,SsBr,ECIndex,ETA_beta,AETA_beta,ETA_beta_s,AETA_beta_s,ETA_beta_ns,AETA_beta_ns,ETA_beta_ns_d,AETA_beta_ns_d,ETA_eta_R,AETA_eta_R,ETA_eta_RL,AETA_eta_RL,ETA_eta_B,AETA_eta_B,ETA_eta_BR,AETA_eta_BR,ETA_epsilon_3,ETA_dBeta,AETA_dBeta,fragCpx,fMF,nHBAcc,nHBDon,IC0,IC1,IC2,IC3,IC4,IC5,TIC0,TIC1,TIC2,TIC3,TIC4,TIC5,SIC0,SIC1,SIC2,SIC3,SIC4,SIC5,BIC0,BIC1,BIC2,BIC3,BIC4,BIC5,CIC0,CIC1,CIC2,CIC3,CIC4,CIC5,MIC0,MIC1,MIC2,MIC3,MIC4,MIC5,ZMIC0,ZMIC1,ZMIC2,ZMIC3,ZMIC4,ZMIC5,Kier1,Kier2,FilterItLogS,LabuteASA,PEOE_VSA1,PEOE_VSA2,PEOE_VSA3,PEOE_VSA4,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,PEOE_VSA9,PEOE_VSA10,PEOE_VSA11,PEOE_VSA12,PEOE_VSA13,SMR_VSA1,SMR_VSA2,SMR_VSA3,SMR_VSA4,SMR_VSA5,SMR_VSA6,SMR_VSA7,SMR_VSA9,SlogP_VSA1,SlogP_VSA2,SlogP_VSA3,SlogP_VSA4,SlogP_VSA5,SlogP_VSA6,SlogP_VSA7,SlogP_VSA8,SlogP_VSA10,SlogP_VSA11,EState_VSA1,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,EState_VSA6,EState_VSA7,EState_VSA8,EState_VSA9,EState_VSA10,VSA_EState1,VSA_EState2,VSA_EState3,VSA_EState4,VSA_EState5,VSA_EState6,VSA_EState7,VSA_EState8,VSA_EState9,MID,AMID,MID_h,AMID_h,MID_C,AMID_C,MID_N,AMID_N,MID_O,AMID_O,MID_X,AMID_X,MPC2,MPC3,MPC4,MPC5,MPC6,MPC7,MPC8,MPC9,MPC10,TMPC10,piPC1,piPC2,piPC3,piPC4,piPC5,piPC6,piPC7,piPC8,piPC9,piPC10,TpiPC10,nRing,n3Ring,n4Ring,n5Ring,n6Ring,n7Ring,nHRing,n3HRing,n5HRing,n6HRing,naRing,n5aRing,n6aRing,n7aRing,naHRing,n5aHRing,n6aHRing,nARing,n3ARing,n4ARing,n5ARing,n6ARing,nAHRing,n3AHRing,n5AHRing,n6AHRing,nFRing,n7FRing,n8FRing,n9FRing,n10FRing,n12FRing,nG12FRing,nFHRing,n8FHRing,n9FHRing,n10FHRing,n12FHRing,nG12FHRing,nFaRing,n8FaRing,n9FaRing,n10FaRing,n12FaRing,nG12FaRing,nFaHRing,n8FaHRing,n9FaHRing,n10FaHRing,n12FaHRing,nG12FaHRing,nFARing,n7FARing,n8FARing,n9FARing,n10FARing,nG12FARing,nFAHRing,n8FAHRing,n9FAHRing,nG12FAHRing,nRot,RotRatio,SLogP,SMR,TopoPSA(NO),TopoPSA,GGI1,GGI2,GGI3,GGI4,GGI5,GGI6,GGI7,GGI8,GGI9,GGI10,JGI1,JGI2,JGI3,JGI4,JGI5,JGI6,JGI7,JGI8,JGI9,JGI10,JGT10,Diameter,Radius,TopoShapeIndex,PetitjeanIndex,VAdjMat,MWC01,MWC02,MWC03,MWC04,MWC05,MWC06,MWC07,MWC08,MWC09,MWC10,TMWC10,SRW02,SRW03,SRW04,SRW05,SRW06,SRW07,SRW08,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,Tc
0,0,0,23.874238,2.347213,4.694426,23.874238,1.256539,3.811874,3.484316,0.183385,1.890126,380.808674,20.042562,6.584151,6

In [15]:
rg_ds.head()

,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,VE1_A,VE2_A,VE3_A,VR1_A,VR2_A,VR3_A,nAromAtom,nAromBond,nAtom,nHeavyAtom,nSpiro,nBridgehead,nHetero,nH,nC,nN,nO,nS,nF,nCl,nBr,nX,ATS0dv,ATS1dv,ATS2dv,ATS3dv,ATS4dv,ATS5dv,ATS6dv,ATS7dv,ATS8dv,ATS0d,ATS1d,ATS2d,ATS3d,ATS4d,ATS5d,ATS6d,ATS7d,ATS8d,ATS0Z,ATS1Z,ATS2Z,ATS3Z,ATS4Z,ATS5Z,ATS6Z,ATS7Z,ATS8Z,AATS0dv,AATS1dv,AATS2dv,AATS0d,AATS1d,AATS2d,AATS0Z,AATS1Z,AATS2Z,ATSC0dv,ATSC1dv,ATSC2dv,ATSC3dv,ATSC4dv,ATSC5dv,ATSC6dv,ATSC7dv,ATSC8dv,ATSC0d,ATSC1d,ATSC2d,ATSC3d,ATSC4d,ATSC5d,ATSC6d,ATSC7d,ATSC8d,ATSC0Z,ATSC1Z,ATSC2Z,ATSC3Z,ATSC4Z,ATSC5Z,ATSC6Z,ATSC7Z,ATSC8Z,AATSC0dv,AATSC1dv,AATSC2dv,AATSC0d,AATSC1d,AATSC2d,AATSC0Z,AATSC1Z,AATSC2Z,MATS1dv,MATS2dv,MATS1d,MATS2d,MATS1Z,MATS2Z,GATS1dv,GATS2dv,GATS1d,GATS2d,GATS1Z,GATS2Z,BCUTdv-1h,BCUTdv-1l,BCUTd-1h,BCUTd-1l,BCUTZ-1h,BCUTZ-1l,BalabanJ,BertzCT,nBonds,nBondsO,nBondsS,nBondsD,nBondsT,nBondsA,nBondsM,nBondsKS,nBondsKD,C1SP1,C2SP1,C1SP2,C2SP2,C3SP2,C1SP3,C2SP3,C3SP3,C4SP3,HybRatio,FCSP3,Xch-3d,Xch-4d,Xch-5d,Xch-6d,Xch-7d,Xch-3dv,Xch-4dv,Xc-3d,Xc-4d,Xc-5d,Xc-6d,Xpc-4d,Xpc-5d,Xpc-6d,Xp-0d,Xp-1d,Xp-2d,Xp-3d,Xp-4d,Xp-5d,Xp-6d,Xp-7d,AXp-0d,AXp-1d,AXp-2d,SZ,MZ,SpAbs_D,SpMax_D,SpDiam_D,SpAD_D,SpMAD_D,LogEE_D,VE1_D,VE2_D,VE3_D,VR1_D,VR2_D,VR3_D,NsCH3,NdCH2,NssCH2,NdsCH,NaaCH,NsssCH,NddC,NtsC,NdssC,NaasC,NaaaC,NssssC,NsNH2,NssNH,NaaNH,NtN,NdsN,NaaN,NsssN,NddsN,NaasN,NssssN,NsOH,NdO,NssO,NaaO,NsF,NssS,NaaS,NdssS,NddssS,NsCl,NsBr,SsCH3,SdCH2,SssCH2,SdsCH,SaaCH,SsssCH,SddC,StsC,SdssC,SaasC,SaaaC,SssssC,SsNH2,SssNH,SaaNH,StN,SdsN,SaaN,SsssN,SddsN,SaasN,SssssN,SsOH,SdO,SssO,SaaO,SsF,SssS,SaaS,SdssS,SddssS,SsCl,SsBr,ECIndex,ETA_beta,AETA_beta,ETA_beta_s,AETA_beta_s,ETA_beta_ns,AETA_beta_ns,ETA_beta_ns_d,AETA_beta_ns_d,ETA_eta_R,AETA_eta_R,ETA_eta_RL,AETA_eta_RL,ETA_eta_B,AETA_eta_B,ETA_eta_BR,AETA_eta_BR,ETA_epsilon_3,ETA_dBeta,AETA_dBeta,fragCpx,fMF,nHBAcc,nHBDon,IC0,IC1,IC2,IC3,IC4,IC5,TIC0,TIC1,TIC2,TIC3,TIC4,TIC5,SIC0,SIC1,SIC2,SIC3,SIC4,SIC5,BIC0,BIC1,BIC2,BIC3,BIC4,BIC5,CIC0,CIC1,CIC2,CIC3,CIC4,CIC5,MIC0,MIC1,MIC2,MIC3,MIC4,MIC5,ZMIC0,ZMIC1,ZMIC2,ZMIC3,ZMIC4,ZMIC5,Kier1,Kier2,FilterItLogS,LabuteASA,PEOE_VSA1,PEOE_VSA2,PEOE_VSA3,PEOE_VSA4,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,PEOE_VSA9,PEOE_VSA10,PEOE_VSA11,PEOE_VSA12,PEOE_VSA13,SMR_VSA1,SMR_VSA2,SMR_VSA3,SMR_VSA4,SMR_VSA5,SMR_VSA6,SMR_VSA7,SMR_VSA9,SlogP_VSA1,SlogP_VSA2,SlogP_VSA3,SlogP_VSA4,SlogP_VSA5,SlogP_VSA6,SlogP_VSA7,SlogP_VSA8,SlogP_VSA10,SlogP_VSA11,EState_VSA1,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,EState_VSA6,EState_VSA7,EState_VSA8,EState_VSA9,EState_VSA10,VSA_EState1,VSA_EState2,VSA_EState3,VSA_EState4,VSA_EState5,VSA_EState6,VSA_EState7,VSA_EState8,VSA_EState9,MID,AMID,MID_h,AMID_h,MID_C,AMID_C,MID_N,AMID_N,MID_O,AMID_O,MID_X,AMID_X,MPC2,MPC3,MPC4,MPC5,MPC6,MPC7,MPC8,MPC9,MPC10,TMPC10,piPC1,piPC2,piPC3,piPC4,piPC5,piPC6,piPC7,piPC8,piPC9,piPC10,TpiPC10,nRing,n3Ring,n4Ring,n5Ring,n6Ring,n7Ring,nHRing,n5HRing,n6HRing,naRing,n5aRing,n6aRing,n7aRing,naHRing,n5aHRing,n6aHRing,nARing,n3ARing,n4ARing,n5ARing,n6ARing,nAHRing,n5AHRing,n6AHRing,nFRing,n7FRing,n9FRing,n10FRing,n12FRing,nG12FRing,nFHRing,n9FHRing,n10FHRing,n12FHRing,nG12FHRing,nFaRing,n9FaRing,n10FaRing,n12FaRing,nG12FaRing,nFaHRing,n9FaHRing,n10FaHRing,n12FaHRing,nG12FaHRing,nFARing,n7FARing,n9FARing,n10FARing,nG12FARing,nFAHRing,n9FAHRing,nG12FAHRing,nRot,RotRatio,SLogP,SMR,TopoPSA(NO),TopoPSA,GGI1,GGI2,GGI3,GGI4,GGI5,GGI6,GGI7,GGI8,GGI9,GGI10,JGI1,JGI2,JGI3,JGI4,JGI5,JGI6,JGI7,JGI8,JGI9,JGI10,JGT10,Diameter,Radius,TopoShapeIndex,PetitjeanIndex,VAdjMat,MWC01,MWC02,MWC03,MWC04,MWC05,MWC06,MWC07,MWC08,MWC09,MWC10,TMWC10,SRW02,SRW03,SRW04,SRW05,SRW06,SRW07,SRW08,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,Rg
0,0,0,48.144017,2.624260,5.064955,48.144017,1.301190,4.527352,4.180237,0.112979,2.738701,3378.775624,91.318260,9.433602,18,18,71,37,0,0,4,34,33,2,0,0,0,0,0,0,364.0,388.0,508.0,529.0,507.0,481.0,468.0,452.0,466.0,222.0,290.0,473.0,573.0,610.0,628.0,657.0,648.0,625.0,1320.0,1584.0,2232.0

In [16]:
ffv_ds.head()

,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,VE1_A,VE2_A,VE3_A,VR1_A,VR2_A,VR3_A,nAromAtom,nAromBond,nAtom,nHeavyAtom,nSpiro,nBridgehead,nHetero,nH,nB,nC,nN,nO,nS,nP,nF,nCl,nBr,nX,ATS0dv,ATS1dv,ATS2dv,ATS3dv,ATS4dv,ATS5dv,ATS6dv,ATS7dv,ATS8dv,ATS0d,ATS1d,ATS2d,ATS3d,ATS4d,ATS5d,ATS6d,ATS7d,ATS8d,ATS0Z,ATS1Z,ATS2Z,ATS3Z,ATS4Z,ATS5Z,ATS6Z,ATS7Z,ATS8Z,AATS0dv,AATS1dv,AATS2dv,AATS0d,AATS1d,AATS2d,AATS0Z,AATS1Z,AATS2Z,ATSC0dv,ATSC1dv,ATSC2dv,ATSC3dv,ATSC4dv,ATSC5dv,ATSC6dv,ATSC7dv,ATSC8dv,ATSC0d,ATSC1d,ATSC2d,ATSC3d,ATSC4d,ATSC5d,ATSC6d,ATSC7d,ATSC8d,ATSC0Z,ATSC1Z,ATSC2Z,ATSC3Z,ATSC4Z,ATSC5Z,ATSC6Z,ATSC7Z,ATSC8Z,AATSC0dv,AATSC1dv,AATSC2dv,AATSC0d,AATSC1d,AATSC2d,AATSC0Z,AATSC1Z,AATSC2Z,MATS1dv,MATS2dv,MATS1d,MATS2d,MATS1Z,MATS2Z,GATS1dv,GATS2dv,GATS1d,GATS2d,GATS1Z,GATS2Z,BCUTdv-1h,BCUTdv-1l,BCUTd-1h,BCUTd-1l,BCUTZ-1h,BCUTZ-1l,BalabanJ,BertzCT,nBonds,nBondsO,nBondsS,nBondsD,nBondsT,nBondsA,nBondsM,nBondsKS,nBondsKD,C1SP1,C2SP1,C1SP2,C2SP2,C3SP2,C1SP3,C2SP3,C3SP3,C4SP3,FCSP3,Xch-3d,Xch-4d,Xch-5d,Xch-6d,Xch-7d,Xch-3dv,Xch-4dv,Xc-3d,Xc-4d,Xc-5d,Xc-6d,Xpc-4d,Xpc-5d,Xpc-6d,Xp-0d,Xp-1d,Xp-2d,Xp-3d,Xp-4d,Xp-5d,Xp-6d,Xp-7d,AXp-0d,AXp-1d,AXp-2d,SZ,MZ,SpAbs_D,SpMax_D,SpDiam_D,SpAD_D,SpMAD_D,LogEE_D,VE1_D,VE2_D,VE3_D,VR1_D,VR2_D,VR3_D,NsssB,NsCH3,NdCH2,NssCH2,NtCH,NdsCH,NaaCH,NsssCH,NddC,NtsC,NdssC,NaasC,NaaaC,NssssC,NsNH2,NdNH,NssNH,NaaNH,NtN,NsssNH,NdsN,NaaN,NsssN,NddsN,NaasN,NssssN,NsOH,NdO,NssO,NaaO,NsF,NssSiH2,NsssSiH,NssssSi,NsssP,NdsssP,NsSH,NdS,NssS,NaaS,NdssS,NddssS,NsCl,NssssGe,NssSe,NaaSe,NsBr,NssssSn,SsssB,SsCH3,SdCH2,SssCH2,StCH,SdsCH,SaaCH,SsssCH,SddC,StsC,SdssC,SaasC,SaaaC,SssssC,SsNH2,SdNH,SssNH,SaaNH,StN,SsssNH,SdsN,SaaN,SsssN,SddsN,SaasN,SssssN,SsOH,SdO,SssO,SaaO,SsF,SssSiH2,SsssSiH,SssssSi,SsssP,SdsssP,SsSH,SdS,SssS,SaaS,SdssS,SddssS,SsCl,SssssGe,SssSe,SaaSe,SsBr,SssssSn,ECIndex,ETA_beta,AETA_beta,ETA_beta_s,AETA_beta_s,ETA_beta_ns,AETA_beta_ns,ETA_beta_ns_d,AETA_beta_ns_d,ETA_dBeta,AETA_dBeta,fragCpx,fMF,nHBAcc,nHBDon,IC0,IC1,IC2,IC3,IC4,IC5,TIC0,TIC1,TIC2,TIC3,TIC4,TIC5,SIC0,SIC1,SIC2,SIC3,SIC4,SIC5,BIC0,BIC1,BIC2,BIC3,BIC4,BIC5,CIC0,CIC1,CIC2,CIC3,CIC4,CIC5,MIC0,MIC1,MIC2,MIC3,MIC4,MIC5,ZMIC0,ZMIC1,ZMIC2,ZMIC3,ZMIC4,ZMIC5,Kier1,Kier2,FilterItLogS,LabuteASA,PEOE_VSA1,PEOE_VSA2,PEOE_VSA3,PEOE_VSA4,PEOE_VSA5,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,PEOE_VSA9,PEOE_VSA10,PEOE_VSA11,PEOE_VSA12,PEOE_VSA13,SMR_VSA1,SMR_VSA2,SMR_VSA3,SMR_VSA4,SMR_VSA5,SMR_VSA6,SMR_VSA7,SMR_VSA9,SlogP_VSA1,SlogP_VSA2,SlogP_VSA3,SlogP_VSA4,SlogP_VSA5,SlogP_VSA6,SlogP_VSA7,SlogP_VSA8,SlogP_VSA10,SlogP_VSA11,EState_VSA1,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,EState_VSA6,EState_VSA7,EState_VSA8,EState_VSA9,EState_VSA10,VSA_EState1,VSA_EState2,VSA_EState3,VSA_EState4,VSA_EState5,VSA_EState6,VSA_EState7,VSA_EState8,VSA_EState9,MID,AMID,MID_h,AMID_h,MID_C,AMID_C,MID_N,AMID_N,MID_O,AMID_O,MID_X,AMID_X,MPC2,MPC3,MPC4,MPC5,MPC6,MPC7,MPC8,MPC9,MPC10,TMPC10,piPC1,piPC2,piPC3,piPC4,piPC5,piPC6,piPC7,piPC8,piPC9,piPC10,TpiPC10,nRing,n3Ring,n4Ring,n5Ring,n6Ring,n7Ring,n8Ring,n10Ring,n11Ring,n12Ring,nG12Ring,nHRing,n3HRing,n4HRing,n5HRing,n6HRing,n7HRing,n8HRing,nG12HRing,naRing,n3aRing,n5aRing,n6aRing,n7aRing,n8aRing,naHRing,n3aHRing,n5aHRing,n6aHRing,nARing,n3ARing,n4ARing,n5ARing,n6ARing,n7ARing,n8ARing,n10ARing,n11ARing,n12ARing,nG12ARing,nAHRing,n3AHRing,n4AHRing,n5AHRing,n6AHRing,n7AHRing,n8AHRing,nG12AHRing,nFRing,n7FRing,n8FRing,n9FRing,n10FRing,n11FRing,n12FRing,nG12FRing,nFHRing,n8FHRing,n9FHRing,n10FHRing,n11FHRing,n12FHRing,nG12FHRing,nFaRing,n8FaRing,n9FaRing,n10FaRing,n12FaRing,nG12FaRing,nFaHRing,n8FaHRing,n9FaHRing,n10FaHRing,n12FaHRing,nG12FaHRing,nFARing,n7FARing,n8FARing,n9FARing,n10FARing,n11FARing,n12FARing,nG12FARing,nFAHRing,n8FAHRing,n9FAHRing,n10FAHRing,n11FAHRing,n12FAHRing,nG12FAHRing,nRot,RotRatio,SLogP,SMR,TopoPSA(NO),TopoPSA,GGI1,GGI2,GGI3,GGI4,GGI5,GGI6,GGI7,GGI8,GGI9,GGI10,JGI1,JGI2,JGI3,JGI4,JGI5,JGI6,JGI7,JGI8,JGI9,JGI10,JGT10,Diameter,Radius,TopoShapeIndex,PetitjeanIndex,VAdjMat,MWC01,MWC02,MWC03,MWC04,MWC05,MWC06,

In [17]:
den_ds.head()

,nAcid,nBase,nAromAtom,nAromBond,nAtom,nHeavyAtom,nSpiro,nBridgehead,nHetero,nH,nC,nN,nO,nS,nF,nCl,nBr,nI,nX,ATS0dv,ATS1dv,ATS2dv,ATS3dv,ATS4dv,ATS5dv,ATS6dv,ATS7dv,ATS8dv,ATS0d,ATS1d,ATS2d,ATS3d,ATS4d,ATS5d,ATS6d,ATS7d,ATS8d,ATS0Z,ATS1Z,ATS2Z,ATS3Z,ATS4Z,ATS5Z,ATS6Z,ATS7Z,ATS8Z,AATS0dv,AATS1dv,AATS2dv,AATS0d,AATS1d,AATS2d,AATS0Z,AATS1Z,AATS2Z,ATSC0dv,ATSC1dv,ATSC2dv,ATSC3dv,ATSC4dv,ATSC5dv,ATSC6dv,ATSC7dv,ATSC8dv,ATSC0d,ATSC1d,ATSC2d,ATSC3d,ATSC4d,ATSC5d,ATSC6d,ATSC7d,ATSC8d,ATSC0Z,ATSC1Z,ATSC2Z,ATSC3Z,ATSC4Z,ATSC5Z,ATSC6Z,ATSC7Z,ATSC8Z,AATSC0dv,AATSC1dv,AATSC2dv,AATSC0d,AATSC1d,AATSC2d,AATSC0Z,AATSC1Z,AATSC2Z,MATS1dv,MATS2dv,MATS1d,MATS2d,MATS1Z,MATS2Z,GATS1dv,GATS2dv,GATS1d,GATS2d,GATS1Z,GATS2Z,BalabanJ,BertzCT,nBonds,nBondsO,nBondsS,nBondsD,nBondsT,nBondsA,nBondsM,nBondsKS,nBondsKD,C1SP1,C2SP1,C1SP2,C2SP2,C3SP2,C1SP3,C2SP3,C3SP3,C4SP3,HybRatio,FCSP3,Xch-3d,Xch-4d,Xch-5d,Xch-6d,Xch-7d,Xch-3dv,Xch-4dv,Xc-3d,Xc-4d,Xc-5d,Xc-6d,Xpc-4d,Xpc-5d,Xpc-6d,Xp-1d,Xp-2d,Xp-3d,Xp-4d,Xp-5d,Xp-6d,Xp-7d,AXp-1d,AXp-2d,SZ,MZ,NsCH3,NdCH2,NssCH2,NdsCH,NaaCH,NsssCH,NddC,NtsC,NdssC,NaasC,NaaaC,NssssC,NsNH2,NssNH,NaaNH,NtN,NdsN,NaaN,NsssN,NddsN,NaasN,NssssN,NsOH,NdO,NssO,NaaO,NsF,NsSH,NdS,NssS,NaaS,NdssS,NddssS,NsCl,NsBr,NsI,SsCH3,SdCH2,SssCH2,SdsCH,SaaCH,SsssCH,SddC,StsC,SdssC,SaasC,SaaaC,SssssC,SsNH2,SssNH,SaaNH,StN,SdsN,SaaN,SsssN,SddsN,SaasN,SssssN,SsOH,SdO,SssO,SaaO,SsF,SsSH,SdS,SssS,SaaS,SdssS,SddssS,SsCl,SsBr,SsI,ECIndex,fragCpx,fMF,nHBAcc,nHBDon,IC0,IC1,IC2,IC3,IC4,IC5,TIC0,TIC1,TIC2,TIC3,TIC4,TIC5,SIC0,SIC1,SIC2,SIC3,SIC4,SIC5,BIC0,BIC1,BIC2,BIC3,BIC4,BIC5,CIC0,CIC1,CIC2,CIC3,CIC4,CIC5,MIC0,MIC1,MIC2,MIC3,MIC4,MIC5,ZMIC0,ZMIC1,ZMIC2,ZMIC3,ZMIC4,ZMIC5,Kier1,Kier2,FilterItLogS,VMcGowan,LabuteASA,PEOE_VSA1,PEOE_VSA2,PEOE_VSA3,PEOE_VSA4,PEOE_VSA5,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,PEOE_VSA9,PEOE_VSA10,PEOE_VSA11,PEOE_VSA12,PEOE_VSA13,SMR_VSA1,SMR_VSA2,SMR_VSA3,SMR_VSA4,SMR_VSA5,SMR_VSA6,SMR_VSA7,SMR_VSA9,SlogP_VSA1,SlogP_VSA2,SlogP_VSA3,SlogP_VSA4,SlogP_VSA5,SlogP_VSA6,SlogP_VSA7,SlogP_VSA8,SlogP_VSA10,SlogP_VSA11,EState_VSA1,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,EState_VSA6,EState_VSA7,EState_VSA8,EState_VSA9,EState_VSA10,VSA_EState1,VSA_EState2,VSA_EState3,VSA_EState4,VSA_EState5,VSA_EState6,VSA_EState7,VSA_EState8,VSA_EState9,MPC2,MPC3,MPC4,MPC5,MPC6,MPC7,MPC8,MPC9,MPC10,TMPC10,piPC1,piPC2,piPC3,piPC4,piPC5,piPC6,piPC7,piPC8,piPC9,piPC10,TpiPC10,apol,bpol,nRing,n3Ring,n4Ring,n5Ring,n6Ring,n7Ring,n8Ring,nG12Ring,nHRing,n3HRing,n5HRing,n6HRing,nG12HRing,naRing,n5aRing,n6aRing,n7aRing,naHRing,n5aHRing,n6aHRing,nARing,n3ARing,n4ARing,n5ARing,n6ARing,n8ARing,nG12ARing,nAHRing,n3AHRing,n5AHRing,n6AHRing,nG12AHRing,nFRing,n7FRing,n9FRing,n10FRing,n12FRing,nG12FRing,nFHRing,n7FHRing,n9FHRing,n10FHRing,n12FHRing,nG12FHRing,nFaRing,n9FaRing,n10FaRing,n12FaRing,nG12FaRing,nFaHRing,n9FaHRing,n10FaHRing,n12FaHRing,nG12FaHRing,nFARing,n7FARing,n9FARing,n10FARing,nG12FARing,nFAHRing,n7FAHRing,n9FAHRing,nG12FAHRing,nRot,RotRatio,SLogP,SMR,TopoPSA(NO),TopoPSA,GGI1,GGI2,GGI3,GGI4,GGI5,GGI6,GGI7,GGI8,GGI9,GGI10,JGI1,JGI2,JGI3,JGI4,JGI5,JGI6,JGI7,JGI8,JGI9,JGI10,JGT10,Diameter,Radius,TopoShapeIndex,PetitjeanIndex,VAdjMat,MWC01,MWC02,MWC03,MWC04,MWC05,MWC06,MWC07,MWC08,MWC09,MWC10,TMWC10,SRW02,SRW03,SRW04,SRW05,SRW06,SRW07,SRW08,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb2,Density
0,0,0,18,18,71,37,0,0,4,34,33,2,0,0,0,0,0,0,0,364.000000,388.0,508.0,529.0,507.0,481.0,468.0,452.0,466.0,222.0,290.0,473.0,573.0,610.0,628.0,657.0,648.0,625.0,1320.0,1584.0,2232.0,2580.0,2738.0,2813.0,2819.0,2811.0,2822.0,5.126761,5.243243,3.791045,3.126761,3.918919,3.529851,18.591549,21.405405,16.656716,199.718310,66.378100,36.193612,33.198373,-46.090260,-73.761357,-81.683793,-83.200952,-57.696092,38.957746,15.142630,9.220591,8.269193,-3.397342,-10.273358,-12.277921,-13.882960,-14.698076,467.661972,19.281889,-60.714144,96.576076,-52.277326,-109.524896,-91.617735,-63.359452,-134.147987,2.812934,0.897001,0.270102,0.548701,0.204630,0.068810,6.586788,0.260566,-0.453

In [18]:
def make_smile_canonical(smile): # To avoid duplicates, for example: canonical '*C=C(*)C' == '*C(=C*)C'
    try:
        mol = Chem.MolFromSmiles(smile)
        canon_smile = Chem.MolToSmiles(mol, canonical=True)
        return canon_smile
    except:
        return np.nan

train['SMILES'] = train['SMILES'].apply(lambda s: make_smile_canonical(s))
test['SMILES'] = test['SMILES'].apply(lambda s: make_smile_canonical(s))

In [19]:
ktg_smiles.rename(columns={'Tg [K]': 'Tg'}, inplace=True)
tg_smiles.rename(columns={'Tg (C)': 'Tg'}, inplace=True)
tc_smiles.rename(columns={'TC_mean': 'Tc'}, inplace=True)
de_smiles.rename(columns={'density(g/cm3)': 'Density'}, inplace=True)
ds1_smiles.rename(columns={'TC_mean': 'Tc'}, inplace=True)

In [20]:
de_smiles['SMILES'] = de_smiles['SMILES'].apply(lambda s: make_smile_canonical(s))
de_smiles = de_smiles[(de_smiles['SMILES'].notnull())&(de_smiles['Density'].notnull())&(de_smiles['Density'] != 'nylon')]
de_smiles['Density'] = de_smiles['Density'].astype('float64')
de_smiles['Density'] -= 0.118

ktg_smiles['Tg'] = ktg_smiles['Tg'] - 273.15

[12:45:47] SMILES Parse Error: syntax error while parsing: *O[Si](*)([R])[R]
[12:45:47] SMILES Parse Error: check for mistakes around position 12:
[12:45:47] *O[Si](*)([R])[R]
[12:45:47] ~~~~~~~~~~~^
[12:45:47] SMILES Parse Error: Failed parsing SMILES '*O[Si](*)([R])[R]' for input: '*O[Si](*)([R])[R]'
[12:45:47] SMILES Parse Error: syntax error while parsing: *NC(=O)c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=O)c3c4
[12:45:47] SMILES Parse Error: check for mistakes around position 28:
[12:45:47] c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=
[12:45:47] ~~~~~~~~~~~~~~~~~~~~^
[12:45:47] SMILES Parse Error: Failed parsing SMILES '*NC(=O)c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=O)c3c4' for input: '*NC(=O)c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=O)c3c4'
[12:45:47] SMILES Parse Error: syntax error while parsing: O=C=N[R1]N=C=O.O[R2]O.O[R3]O
[12:45:47] SMILES Parse Error: check for mistakes around position 7:
[12:45:47] O=C=N[R1]N=C=O.O[R2]O.O[R3]O
[12:45:47] ~~~~~~^
[12:45:47] SMILES Parse Error: F

In [21]:
def preprocessing(df):
    desc_names = [desc[0] for desc in Descriptors.descList if desc[0] not in useless_cols]
    descriptors = [compute_all_descriptors(smi) for smi in df['SMILES'].to_list()]

    graph_feats = {'graph_diameter': [], 'avg_shortest_path': [], 'num_cycles': []}
    for smile in df['SMILES']:
         compute_graph_features(smile, graph_feats)
        
    result = pd.concat(
        [
            pd.DataFrame(descriptors, columns=desc_names),
            pd.DataFrame(graph_feats)
        ],
        axis=1
    )

    result = result.replace([-np.inf, np.inf], np.nan)
    return result

# Feature Extraction 

In [22]:
# SMILESを正規化（canonical化）
# 重複する SMILES を集約して平均 target 値に
# df_train に存在するが欠損している target に、df_extra から値を補完

def add_extra_data(df_train, df_extra, target):
    n_samples_before = len(df_train[df_train[target].notnull()])
    
    df_extra['SMILES'] = df_extra['SMILES'].apply(lambda s: make_smile_canonical(s))
    df_extra = df_extra.groupby('SMILES', as_index=False)[target].mean()
    cross_smiles = set(df_extra['SMILES']) & set(df_train['SMILES'])
    unique_smiles_extra = set(df_extra['SMILES']) - set(df_train['SMILES'])

    # Make priority target value from competition's df
    for smile in df_train[df_train[target].notnull()]['SMILES'].tolist():
        if smile in cross_smiles:
            cross_smiles.remove(smile)

    # Imput missing values for competition's SMILES
    for smile in cross_smiles:
        df_train.loc[df_train['SMILES']==smile, target] = df_extra[df_extra['SMILES']==smile][target].values[0]
    
    df_train = pd.concat([df_train, df_extra[df_extra['SMILES'].isin(unique_smiles_extra)]], axis=0).reset_index(drop=True)

    n_samples_after = len(df_train[df_train[target].notnull()])
    print(f'\nFor target "{target}" added {n_samples_after-n_samples_before} new samples!')
    print(f'New unique SMILES: {len(unique_smiles_extra)}')
    return df_train

train = add_extra_data(train, tc_smiles, 'Tc')
train = add_extra_data(train, tg_smiles, 'Tg')
train = add_extra_data(train, ktg_smiles, 'Tg')
train = add_extra_data(train, de_smiles, 'Density')

train = add_extra_data(train, ds1_smiles, 'Tc')
#train = add_extra_data(train, ds2_smiles, 'Density')
train = add_extra_data(train, ds3_smiles, 'Tg')
train = add_extra_data(train, ds4_smiles, 'FFV')


For target "Tc" added 129 new samples!
New unique SMILES: 129

For target "Tg" added 151 new samples!
New unique SMILES: 136

For target "Tg" added 499 new samples!
New unique SMILES: 499

For target "Density" added 634 new samples!
New unique SMILES: 524

For target "Tc" added 0 new samples!
New unique SMILES: 0

For target "Tg" added 0 new samples!
New unique SMILES: 0

For target "FFV" added 862 new samples!
New unique SMILES: 819


In [23]:
def compute_all_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [None] * len(desc_names)
    return [desc[1](mol) for desc in Descriptors.descList if desc[0] not in useless_cols]

def compute_graph_features(smiles, graph_feats):
    mol = Chem.MolFromSmiles(smiles)
    adj = rdmolops.GetAdjacencyMatrix(mol)
    G = nx.from_numpy_array(adj)

    graph_feats['graph_diameter'].append(nx.diameter(G) if nx.is_connected(G) else 0)
    graph_feats['avg_shortest_path'].append(nx.average_shortest_path_length(G) if nx.is_connected(G) else 0)
    graph_feats['num_cycles'].append(len(list(nx.cycle_basis(G))))

train = pd.concat([train, preprocessing(train)], axis=1)
test = pd.concat([test, preprocessing(test)], axis=1)

# Find constant columns for each target
all_features = train.columns[7:].tolist()
features = {}
for target in CFG.TARGETS:
    const_descs = []
    for col in train.columns.drop(CFG.TARGETS):
        if train[train[target].notnull()][col].nunique() == 1:
            const_descs.append(col)
    features[target] = [f for f in all_features if f not in const_descs]

print(train.shape)
train['Ipc']=np.log10(train['Ipc'])  
for n in train.columns[7:]:
    train[n]=train[n].replace(-np.inf,np.nan)
    train[n]=train[n].replace(np.inf,np.nan)    
    train[n].fillna(train[n].mean())
  
print(train.shape)
test['Ipc']=np.log10(test['Ipc'])
for n in test.columns[7:]:
    train[n]=train[n].replace(-np.inf,np.nan)
    train[n]=train[n].replace(np.inf,np.nan)      
    test[n].fillna(train[n].mean())

(10080, 158)
(10080, 158)


In [24]:
train

,id,SMILES,Tg,FFV,Tc,Density,Rg,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,FpDensityMorgan1,AvgIpc,BalabanJ,Ipc,Kappa2,PEOE_VSA1,PEOE_VSA10,PEOE_VSA11,PEOE_VSA12,PEOE_VSA13,PEOE_VSA14,PEOE_VSA2,PEOE_VSA3,PEOE_VSA4,PEOE_VSA5,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,PEOE_VSA9,SMR_VSA1,SMR_VSA10,SMR_VSA2,SMR_VSA3,SMR_VSA4,SMR_VSA5,SMR_VSA6,SMR_VSA7,SMR_VSA9,SlogP_VSA1,SlogP_VSA10,SlogP_VSA11,SlogP_VSA12,SlogP_VSA2,SlogP_VSA3,SlogP_VSA4,SlogP_VSA5,SlogP_VSA7,SlogP_VSA8,TPSA,EState_VSA1,EState_VSA10,EState_VSA11,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,EState_VSA6,EState_VSA7,EState_VSA8,EState_VSA9,VSA_EState1,VSA_EState10,VSA_EState2,VSA_EState3,VSA_EState4,VSA_EState5,VSA_EState7,VSA_EState8,VSA_EState9,FractionCSP3,NHOHCount,NumAliphaticCarbocycles,NumAliphaticHeterocycles,NumAliphaticRings,NumAmideBonds,NumAromaticHeterocycles,NumAtomStereoCenters,NumBridgeheadAtoms,NumHAcceptors,NumHeteroatoms,NumHeterocycles,NumRotatableBonds,NumSaturatedCarbocycles,NumSaturatedHeterocycles,NumSaturatedRings,NumSpiroAtoms,NumUnspecifiedAtomStereoCenters,RingCount,MolLogP,fr_Al_COO,fr_Al_OH,fr_Al_OH_noTert,fr_ArN,fr_Ar_COO,fr_Ar_N,fr_Ar_NH,fr_Ar_OH,fr_COO,fr_C_S,fr_HOCCN,fr_Imine,fr_NH0,fr_NH1,fr_NH2,fr_N_O,fr_Ndealkylation1,fr_Ndealkylation2,fr_SH,fr_aldehyde,fr_alkyl_carbamate,fr_alkyl_halide,fr_allylic_oxid,fr_amidine,fr_aniline,fr_aryl_methyl,fr_azide,fr_azo,fr_bicyclic,fr_ester,fr_ether,fr_furan,fr_guanido,fr_hdrzine,fr_hdrzone,fr_imidazole,fr_imide,fr_isocyan,fr_ketone,fr_ketone_Topliss,fr_lactone,fr_methoxy,fr_morpholine,fr_nitro,fr_nitro_arom_nonortho,fr_oxazole,fr_oxime,fr_para_hydroxylation,fr_phos_acid,fr_piperdine,fr_piperzine,fr_priamide,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea,graph_diameter,avg_shortest_path,num_cycles
0,87817.0,*CC(*)c1ccccc1C(=O)OCCCCCC,NaN,0.374645,0.205667,0.932,NaN,12.144536,0.105927,-0.105927,0.500278,13.705882,232.323,1.411765,2.456411,2.563477,4.261290,5.852071,0.000000,0.000000,0.0,0.000000,0.000000,103.451541,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,9.531400,5.969305,0.0,0.000000,0.000000,45.951583,6.606882,35.392371,0.000000,0.000000,0.000000,0.000000,0.000000,12.576187,4.736863,0.000000,54.949285,6.923737,0.000000,26.30,0.000000,4.794537,0.0,5.969305,6.606882,24.825916,25.328832,0.000000,12.132734,19.056471,4.736863,6.120445,0.000000,12.144536,0.000000,2.105648,-0.105927,5.344954,4.148954,0.000000,0.533333,0,0,0,0,0,0,1,0,2,4,0,8,0,0,0,0,1,1,3.98170,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,12,4.736842,1
1,106919.0,*Nc1ccc([C@H](CCC)c2ccc(C3(c4ccc([C@@H](CCC)c5...,NaN,0.370410,NaN,NaN,NaN,3.523412,0.098918,0.098918,0.125364,16.777778,598.919,0.577778,3.135512,1.451540,10.636376,14.715582,0.000000,0.000000,0.0,0.000000,0.000000,222.907896,0.000000,0.000000,0.0,0.0,32.607024,18.759549,0.000000,0.000000,0.000000,11.374773,0.0,0.000000,5.917906,115.071874,11.467335,130.442582,0.000000,11.467335,11.374773,0.000000,0.000000,0.000000,5.414990,5.917906,143.037592,0.000000,0.000000,24.06,0.000000,0.000000,0.0,5.414990,11.835812,5.917906,121.805341,0.000000,0.000000,129.300420,0.000000,0.326442,0.000000,0.000000,0.000000,18.579143,1.794202,15.456662,6.971589,0.000000,0.441860,2,1,0,1,0,0,2,0,2,4,0,16,1,0,1,0,0,5,12.35960,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,22,8.364477,5
2,388772.0,*Oc1ccc(S(=O)(=O)c2ccc(Oc3ccc(C4(c5ccc(Oc6ccc(...,NaN,0.378860,NaN,NaN,NaN,13.714745,0.107441,-3.829434,0.092387,16.301370,1003.207,0.397260,3.440031,0.743739,16.833145,19.899082,9.473726,22.998047,0.0,9.837253,0.000000,222.531582,0.000000,8.417797,0.0,0.0,43.527933,96.764951,5.414990,9.790967,40.947247,43.676159,0.0,0.000000,0.000000,76.363497,0.000000,227.523761,40.246583,25.383483,0.000000,40.246583,0.000000,22.618839,29.88403

# Data Preparation For Model Training 

In [25]:
# We'll separate train to be one model for each target variable.
t_1=train[['SMILES','Tg']].copy()
t_2=train[['SMILES','FFV']].copy()
t_3=train[['SMILES','Tc']].copy()
t_4=train[['SMILES','Density']].copy()
t_5=train[['SMILES','Rg']].copy()

# We will drop the rows with missing values related to that target after separation.
#This is important , dropping them beforehand would result Null for all data.
t_1.dropna(inplace=True)
t_2.dropna(inplace=True)
t_3.dropna(inplace=True)
t_4.dropna(inplace=True)
t_5.dropna(inplace=True)

In [26]:
train=train.drop(['id','Tg','FFV','Tc','Density','Rg'],axis=1)
test=test.drop(['id','SMILES'],axis=1)

In [27]:
tg=t_1.merge(train,on='SMILES',how='left')
ffv=t_2.merge(train,on='SMILES',how='left')
tc=t_3.merge(train,on='SMILES',how='left')
density=t_4.merge(train,on='SMILES',how='left')
rg=t_5.merge(train,on='SMILES',how='left')

In [28]:
for i in (tg,tc,density,ffv,rg):
    i.drop('SMILES',axis=1,inplace=True)
    i.dropna(inplace=True)

In [29]:
tg.head()

,Tg,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,FpDensityMorgan1,AvgIpc,BalabanJ,Ipc,Kappa2,PEOE_VSA1,PEOE_VSA10,PEOE_VSA11,PEOE_VSA12,PEOE_VSA13,PEOE_VSA14,PEOE_VSA2,PEOE_VSA3,PEOE_VSA4,PEOE_VSA5,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,PEOE_VSA9,SMR_VSA1,SMR_VSA10,SMR_VSA2,SMR_VSA3,SMR_VSA4,SMR_VSA5,SMR_VSA6,SMR_VSA7,SMR_VSA9,SlogP_VSA1,SlogP_VSA10,SlogP_VSA11,SlogP_VSA12,SlogP_VSA2,SlogP_VSA3,SlogP_VSA4,SlogP_VSA5,SlogP_VSA7,SlogP_VSA8,TPSA,EState_VSA1,EState_VSA10,EState_VSA11,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,EState_VSA6,EState_VSA7,EState_VSA8,EState_VSA9,VSA_EState1,VSA_EState10,VSA_EState2,VSA_EState3,VSA_EState4,VSA_EState5,VSA_EState7,VSA_EState8,VSA_EState9,FractionCSP3,NHOHCount,NumAliphaticCarbocycles,NumAliphaticHeterocycles,NumAliphaticRings,NumAmideBonds,NumAromaticHeterocycles,NumAtomStereoCenters,NumBridgeheadAtoms,NumHAcceptors,NumHeteroatoms,NumHeterocycles,NumRotatableBonds,NumSaturatedCarbocycles,NumSaturatedHeterocycles,NumSaturatedRings,NumSpiroAtoms,NumUnspecifiedAtomStereoCenters,RingCount,MolLogP,fr_Al_COO,fr_Al_OH,fr_Al_OH_noTert,fr_ArN,fr_Ar_COO,fr_Ar_N,fr_Ar_NH,fr_Ar_OH,fr_COO,fr_C_S,fr_HOCCN,fr_Imine,fr_NH0,fr_NH1,fr_NH2,fr_N_O,fr_Ndealkylation1,fr_Ndealkylation2,fr_SH,fr_aldehyde,fr_alkyl_carbamate,fr_alkyl_halide,fr_allylic_oxid,fr_amidine,fr_aniline,fr_aryl_methyl,fr_azide,fr_azo,fr_bicyclic,fr_ester,fr_ether,fr_furan,fr_guanido,fr_hdrzine,fr_hdrzone,fr_imidazole,fr_imide,fr_isocyan,fr_ketone,fr_ketone_Topliss,fr_lactone,fr_methoxy,fr_morpholine,fr_nitro,fr_nitro_arom_nonortho,fr_oxazole,fr_oxime,fr_para_hydroxylation,fr_phos_acid,fr_piperdine,fr_piperzine,fr_priamide,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea,graph_diameter,avg_shortest_path,num_cycles
0,208.639749,11.248449,0.023312,-0.334079,0.435492,13.615385,185.183,1.384615,2.210448,3.709964,3.151338,3.993205,0.0,0.0,0.0,0.0,0.0,74.906467,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,14.383612,18.100520,0.0,16.367245,0.0,12.965578,13.089513,0.000000,0.000000,16.367245,0.0,0.000000,0.000000,37.231874,14.383612,0.0,6.923737,0.000000,0.000000,87.30,0.0,14.383612,0.0,37.231874,0.000000,0.000000,0.000000,6.923737,0.000000,16.367245,0.000000,-0.357391,0.000000,33.004449,4.941399,3.000789,-0.488941,0.133398,1.818204,0.000000,0.571429,3,0,0,0,2,0,1,0,4,8,0,6,0,0,0,0,1,0,-2.2629,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,4.285714,0
1,-41.266724,2.142025,0.580086,0.580086,0.318231,12.571429,268.538,0.714286,2.306121,2.828262,3.566187,10.902614,0.0,0.0,0.0,0.0,0.0,106.749724,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,44.042496,0.0,0.000000,0.0,45.448667,17.258561,0.000000,0.000000,0.000000,0.0,0.000000,44.042496,17.258561,0.000000,0.0,38.524930,6.923737,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,55.783491,6.923737,0.000000,44.042496,0.000000,1.277360,2.142025,0.000000,0.000000,0.000000,4.148997,8.837643,1.611538,6.315771,1.000000,0,0,0,0,0,0,0,0,4,6,0,13,0,0,0,0,0,0,5.5217,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,15,5.666667,0
2,-17.282022,2.408148,0.540428,0.540428,0.448030,13.600000,138.254,1.000000,2.098391,2.813864,2.689242,5.519191,0.0,0.0,0.0,0.0,0.0,64.524444,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,51.869489,0.000000,12.654956,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,44.945751,6.923737,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,38.524930,13.344559,0.000000,12.654956,0.000000,1.252280,0.000000,0.000000,0.000000,0.000000,0.000000,12.721612,4.026108,0.000000,0.800000,0,0,0,0,0,0,0,0,0,2,0,8,0,0,0,0,0,0,3.7477,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [30]:
desc_test.head()

ABC  \
0  module 'numpy' has no attribute 'float'.\n`np....   
1  module 'numpy' has no attribute 'float'.\n`np....   
2  module 'numpy' has no attribute 'float'.\n`np....   

                                               ABCGG  nAcid  nBase    SpAbs_A  \
0  module 'numpy' has no attribute 'float'.\n`np....      0      0  51.369068   
1  module 'numpy' has no attribute 'float'.\n`np....      0      0  53.698919   
2  module 'numpy' has no attribute 'float'.\n`np....      0      0  60.526875   

    SpMax_A  SpDiam_A     SpAD_A   SpMAD_A   LogEE_A     VE1_A     VE2_A  \
0  2.608687  5.217373  51.369068  1.252904  4.633213  4.054325  0.098886   
1  2.429583  4.859165  53.698919  1.309730  4.643161  4.685471  0.114280   
2  2.564154  4.983326  60.526875  1.315802  4.764144  4.942681  0.107450   

      VE3_A          VR1_A         VR2_A      VR3_A  nAromAtom  nAromBond  \
0  2.810771  138187.404971   3370.424511  13.247353         24         24   
1  2.955453     981.187389     23.931400   8.299750         30         30   
2  3.123964  637481.849542  13858.301077  14.891337         24         24   

   nAtom  nHeavyAtom  nSpiro  nBridgehead  nHetero  nH  nB  nC  nN  nO  nS  \
0     59          41       0            0       12  18   0  29   2   2   0   
1     67          41       0            0        6  26   0  35   0   4   0   
2     76          46       0            0       10  30   0  36   2   6   0   

   nP  nF  nCl  nBr  nI  nX  ATS0dv  ATS1dv  ATS2dv  ATS3dv  ATS4dv  ATS5dv  \
0   0   6    0    0   0   6   754.0   647.0  1100.0  1133.0  1481.0   988.0   
1   0   0    0    0   0   0   534.0   554.0   760.0   789.0   700.0   672.0   
2   0   0    0    0   0   0   648.0   655.0   974.0  1023.0   953.0   805.0   

   ATS6dv  ATS7dv  ATS8dv  ATS0d  ATS1d  ATS2d  ATS3d  ATS4d  ATS5d  ATS6d  \
0   788.0   662.0   564.0  234.0  291.0  429.0  490.0  466.0  427.0  353.0   
1   648.0   576.0   550.0  244.0  303.0  463.0  534.0  512.0  485.0  465.0   
2   797.0   722.0   743.0  276.0  355.0  550.0  599.0  573.0  552.0  534.0   

   ATS7d  ATS8d                                            ATS0s  \
0  311.0  294.0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  414.0  361.0  missing intrinsic state for ['*'] (ATS0s/Props)   
2  515.0  470.0  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS1s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS2s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS3s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS4s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS5s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS6s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS7s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS8s   ATS0Z   ATS1Z   ATS2Z  \
0  missing intrinsic state for ['*'] (ATS0s/Props)  1774.0  1789.0  2888

# Model 

In [31]:
# Let’s define a reusable function to train and evaluate our machine learning model.

def model(train_d,test_d,model,target,submission=False):
    # We divide the data into training and validation sets for model evaluation
    train_cols = set(train_d.columns) - {target}
    test_cols = set(test_d.columns)
   # Intersect the feature columns
    common_cols = list(train_cols & test_cols)
    X=train_d[common_cols].copy()
    y=train_d[target].copy()
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=10)

    Model=model()
    if submission==False:
       Model.fit(X_train,y_train)
       y_pred=Model.predict(X_test)
       return mean_absolute_error(y_pred,y_test)         # We assess our model performance using MAE metric
    if submission==True:
       Model.fit(X,y)
       submission=Model.predict(test_d[common_cols].copy())
       return submission

# **Model Evaluation

In [32]:
from sklearn.ensemble import (
    RandomForestRegressor,
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    HistGradientBoostingRegressor
)
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler

In [33]:

models_to_compare = {
    'RandomForest': lambda: RandomForestRegressor(
        n_estimators=500,
        max_depth=12,
        min_samples_leaf=2,
        max_features='sqrt',
        n_jobs=-1,
        random_state=SEED
    ),
    'ExtraTrees': lambda: ExtraTreesRegressor(
        n_estimators=500,
        max_depth=12,
        min_samples_leaf=2,
        max_features='sqrt',
        n_jobs=-1,
        random_state=SEED
    ),
    'GradientBoosting': lambda: GradientBoostingRegressor(
        # compromise between the Density (shallow, lr=0.1) and Rg (deeper, lr=0.01) configs
        n_estimators=900,
        learning_rate=0.03,
        max_depth=5,
        subsample=0.8,
        max_features='sqrt',
        random_state=SEED
    ),
    'HistGradientBoosting': lambda: HistGradientBoostingRegressor(
        max_iter=1000,
        learning_rate=0.05,
        max_leaf_nodes=31,
        early_stopping='auto',
        random_state=SEED
    ),
    'XGBoost': lambda: XGBRegressor(
        # tuned for FFV
        n_estimators=800,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.7,
        reg_alpha=0.0,
        reg_lambda=1,
        tree_method='auto',   # change to 'gpu_hist' if GPU-compatible wheel and GPU available
        random_state=SEED,
        verbosity=0,
        n_jobs=-1
    ),
    'LightGBM': lambda: LGBMRegressor(
        # tuned for Tc
        n_estimators=500,
        learning_rate=0.01,
        num_leaves=64,
        subsample=0.6,
        colsample_bytree=0.5,
        reg_alpha=0.001,
        reg_lambda=1,
        random_state=SEED,
        n_jobs=-1,
        verbose=-1
    ),
    'CatBoost': lambda: CatBoostRegressor(
        # tuned for Tg
        iterations=1000,
        learning_rate=0.03,
        depth=6,
        l2_leaf_reg=3,
        early_stopping_rounds=50,
        random_seed=SEED,
        verbose=0,
        allow_writing_files=False
    ),
    'RandomForest2': lambda: RandomForestRegressor(),
    'ExtraTrees2': lambda: ExtraTreesRegressor(),
    'GradientBoosting2': lambda: GradientBoostingRegressor(),
    'HistGradientBoosting2': lambda: HistGradientBoostingRegressor(),
    'XGBoost2': lambda: XGBRegressor(verbosity=0),
    'LightGBM2': lambda: LGBMRegressor(verbose=-1),
    'CatBoost2': lambda: CatBoostRegressor(verbose=0),
}

In [34]:
scaler = StandardScaler()
tg["Tg"] = scaler.fit_transform(np.array(tg["Tg"]).reshape(-1, 1))

for name, model_init in models_to_compare.items():
    try:
        score = model(tg_ds, test, model_init, 'Tg', submission=False)
        print(f'{name:<21}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<21}: Failed with error: {e}')

RandomForest         : MAE = 35.50049
ExtraTrees           : MAE = 47.21965
GradientBoosting     : MAE = 31.52255
HistGradientBoosting : MAE = 28.20003
XGBoost              : MAE = 27.12180
LightGBM             : MAE = 33.32396
CatBoost             : MAE = 34.31221
RandomForest2        : MAE = 30.23503
ExtraTrees2          : MAE = 26.15506
GradientBoosting2    : MAE = 42.48231
HistGradientBoosting2: MAE = 33.54024
XGBoost2             : MAE = 30.50371
LightGBM2            : MAE = 33.37392
CatBoost2            : MAE = 31.52562


In [35]:

for name, model_cls in models_to_compare.items():
    try:
        score = model(ffv_ds, test, model_cls, 'FFV', submission=False)
        print(f'{name:<21}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<21}: Failed with error: {e}')

RandomForest         : MAE = 0.00922
ExtraTrees           : MAE = 0.01263
GradientBoosting     : MAE = 0.00771
HistGradientBoosting : MAE = 0.00705
XGBoost              : MAE = 0.00693
LightGBM             : MAE = 0.00815
CatBoost             : MAE = 0.00829
RandomForest2        : MAE = 0.00836
ExtraTrees2          : MAE = 0.00740
GradientBoosting2    : MAE = 0.01080
HistGradientBoosting2: MAE = 0.00831
XGBoost2             : MAE = 0.00794
LightGBM2            : MAE = 0.00833
CatBoost2            : MAE = 0.00777


In [36]:

for name, model_cls in models_to_compare.items():
    try:
        score = model(tc_ds, test, model_cls, 'Tc', submission=False)
        print(f'{name:<21}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<21}: Failed with error: {e}')

RandomForest         : MAE = 0.04136
ExtraTrees           : MAE = 0.04262
GradientBoosting     : MAE = 0.04037
HistGradientBoosting : MAE = 0.04252
XGBoost              : MAE = 0.04084
LightGBM             : MAE = 0.04007
CatBoost             : MAE = 0.03969
RandomForest2        : MAE = 0.04155
ExtraTrees2          : MAE = 0.04099
GradientBoosting2    : MAE = 0.04113
HistGradientBoosting2: MAE = 0.04123
XGBoost2             : MAE = 0.04139
LightGBM2            : MAE = 0.03997
CatBoost2            : MAE = 0.03953


In [37]:

for name, model_cls in models_to_compare.items():
    try:
        score = model(den_ds, test, model_cls, 'Density', submission=False)
        print(f'{name:<21}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<21}: Failed with error: {e}')

RandomForest         : MAE = 0.07823
ExtraTrees           : MAE = 0.08130
GradientBoosting     : MAE = 0.07974
HistGradientBoosting : MAE = 0.09041
XGBoost              : MAE = 0.08947
LightGBM             : MAE = 0.07872
CatBoost             : MAE = 0.07226
RandomForest2        : MAE = 0.08467
ExtraTrees2          : MAE = 0.09338
GradientBoosting2    : MAE = 0.07602
HistGradientBoosting2: MAE = 0.08103
XGBoost2             : MAE = 0.09058
LightGBM2            : MAE = 0.08212
CatBoost2            : MAE = 0.07582


In [38]:

for name, model_cls in models_to_compare.items():
    try:
        score = model(rg_ds, test, model_cls, 'Rg', submission=False)
        print(f'{name:<21}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<21}: Failed with error: {e}')

RandomForest         : MAE = 2.03318
ExtraTrees           : MAE = 2.18464
GradientBoosting     : MAE = 1.93300
HistGradientBoosting : MAE = 1.89509
XGBoost              : MAE = 1.89993
LightGBM             : MAE = 1.85964
CatBoost             : MAE = 1.95838
RandomForest2        : MAE = 1.80893
ExtraTrees2          : MAE = 1.91173
GradientBoosting2    : MAE = 1.95230
HistGradientBoosting2: MAE = 1.85699
XGBoost2             : MAE = 2.00328
LightGBM2            : MAE = 1.95593
CatBoost2            : MAE = 1.92124


# Final Model For Submission

In [39]:

models = {
    'RandomForest': lambda: RandomForestRegressor(
        n_estimators=500,
        max_depth=12,
        min_samples_leaf=2,
        max_features='sqrt',
        n_jobs=-1,
        random_state=SEED
    ),
    'ExtraTrees': lambda: ExtraTreesRegressor(
        n_estimators=500,
        max_depth=12,
        min_samples_leaf=2,
        max_features='sqrt',
        n_jobs=-1,
        random_state=SEED
    ),
    'GradientBoosting': lambda: GradientBoostingRegressor(
        # compromise between the Density (shallow, lr=0.1) and Rg (deeper, lr=0.01) configs
        n_estimators=900,
        learning_rate=0.03,
        max_depth=5,
        subsample=0.8,
        max_features='sqrt',
        random_state=SEED
    ),
    'HistGradientBoosting': lambda: HistGradientBoostingRegressor(
        max_iter=1000,
        learning_rate=0.05,
        max_leaf_nodes=31,
        early_stopping='auto',
        random_state=SEED
    ),
    'XGBoost': lambda: XGBRegressor(
        # tuned for FFV
        n_estimators=800,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.7,
        reg_alpha=0.0,
        reg_lambda=1,
        tree_method='auto',   # change to 'gpu_hist' if GPU-compatible wheel and GPU available
        random_state=SEED,
        verbosity=0,
        n_jobs=-1
    ),
    'LightGBM': lambda: LGBMRegressor(
        # tuned for Tc
        n_estimators=500,
        learning_rate=0.01,
        num_leaves=64,
        subsample=0.6,
        colsample_bytree=0.5,
        reg_alpha=0.001,
        reg_lambda=1,
        random_state=SEED,
        n_jobs=-1,
        verbose=-1
    ),
    'CatBoost': lambda: CatBoostRegressor(
        # tuned for Tg
        iterations=1000,
        learning_rate=0.03,
        depth=6,
        l2_leaf_reg=3,
        early_stopping_rounds=50,
        random_seed=SEED,
        verbose=0,
        allow_writing_files=False
    ),
    'RandomForest2': lambda: RandomForestRegressor(),
    'ExtraTrees2': lambda: ExtraTreesRegressor(),
    'GradientBoosting2': lambda: GradientBoostingRegressor(),
    'HistGradientBoosting2': lambda: HistGradientBoostingRegressor(),
    'XGBoost2': lambda: XGBRegressor(verbosity=0),
    'LightGBM2': lambda: LGBMRegressor(verbose=-1),
    'CatBoost2': lambda: CatBoostRegressor(verbose=0),
}
'''
11297.2s	273	=== Best models summary ===
11297.2s	274	Tg: CatBoost  CV_MAE=42.369263  params={'learning_rate': 0.03, 'l2_leaf_reg': 3, 'iterations': 1000, 'depth': 6}
11297.2s	275	FFV: XGBoost  CV_MAE=0.006229  params={'subsample': 0.8, 'reg_lambda': 1, 'reg_alpha': 0.0, 'n_estimators': 800, 'max_depth': 6, 'learning_rate': 0.05, 'colsample_bytree': 0.7}
11297.2s	276	Tc: LightGBM  CV_MAE=0.032098  params={'subsample': 0.6, 'reg_lambda': 1, 'reg_alpha': 0.001, 'num_leaves': 64, 'n_estimators': 500, 'learning_rate': 0.01, 'colsample_bytree': 0.5}
11297.2s	277	Density: GradientBoosting  CV_MAE=0.065598  params={'subsample': 0.8, 'n_estimators': 800, 'max_features': 'sqrt', 'max_depth': 3, 'learning_rate': 0.1}
11297.2s	278	Rg: GradientBoosting  CV_MAE=1.717341  params={'subsample': 0.8, 'n_estimators': 1000, 'max_features': 'sqrt', 'max_depth': 5, 'learning_rate': 0.01}
'''

def predict(train, test, model1, model2, target):
    return (model(train, test, model1, target, submission=True)*0.6+
            model(train, test, model2, target, submission=True)*0.4)/1
sub = {
    'id': ID,
    'Tg': predict(tg_ds, desc_test, models['XGBoost'], models['HistGradientBoosting'], 'Tg'),
    'FFV': predict(ffv_ds, desc_test, models['XGBoost'], models['HistGradientBoosting'], 'FFV'),
    'Tc': predict(tc_ds, desc_test, models['LightGBM2'], models['CatBoost'], 'Tc'),
    'Density': predict(den_ds, desc_test, models['CatBoost'], models['CatBoost2'], 'Density'),
    'Rg': predict(rg_ds, desc_test, models['RandomForest2'], models['LightGBM'], 'Rg'),
}

In [40]:
submission=pd.DataFrame(sub)

In [41]:
submission

,id,Tg,FFV,Tc,Density,Rg
0,1109053969,155.506966,0.373338,0.181503,1.138853,19.640892
1,1422188626,152.619809,0.377398,0.240602,1.102433,20.789982
2,2032016830,142.653600,0.351644,0.231091,1.108569,21.669719


In [42]:
submission.to_csv('submission.csv',index=False)